In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings(action='ignore')

# scikit-learn
import sklearn

# 수치형 변환
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
# PCA (차원 축소 : 주성분분석)
from sklearn.decomposition import PCA

# 문제지, 답안지 분리
from sklearn.model_selection import train_test_split

# 분류 모델
from sklearn.linear_model import LogisticRegression # 분류 모델 (회귀 모델 아님)
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier # Tree : 나무
from sklearn.ensemble import RandomForestClassifier # Forest : 나무가 모여서 숲이 됨

# 정규화 / 스케일러
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

# 교차 검증
from sklearn.model_selection import KFold, StratifiedKFold, cross_validate, GridSearchCV

# 분류 평가 매트릭스
from sklearn.metrics import accuracy_score

# --- 필요한 라이브러리 추가 임포트 ---
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, log_loss
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import RFE

# matplotlib 차트 스타일을 seaborn 스타일로 설정
sns.set()

# ----------- 차트 관련 속성 (한글처리, 그리드) -----------
plt.rcParams['font.family']= 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

#-------------------- 주피터 , 출력결과 넓이 늘리기 ---------------
# from IPython.core.display import display, HTML
from IPython.display import display, HTML

display(HTML("<style>.container{width:100% !important;}</style>"))
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('max_colwidth', None)

In [15]:
# 데이터 불러오기
train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')

# 점수 및 csv 출력 함수 제작

In [16]:
# ----------------------------------------------------
# 분류용 최종 myscore 함수
# ----------------------------------------------------

def cls_score(defi=None, model=None, name):
    # id 따로 빼기
    test_ids = test_df['id']

    # 학습에는 id 필요없으니 id 없애기
    trdf = train_df.drop(columns=['id'])
    tdf = test_df.drop(columns=['id'])

    # 컬럼 변환
    trdf.columns = ['bl', 'rf', 'hl', 'hs', 'cl', 'tp']
    tdf.columns = ['bl', 'rf', 'hl', 'hs', 'cl']
    
    # 데이터 전처리용 def가 있을 시 적용
    if defi is not None :
        for Dd in defi :
            trdf = Dd(trdf)
            test_df = Dd(test_df)

    # 결과용 데이터 제작
    X_train = trdf.drop(columns=['tp'])
    X_test = tdf

    # 라벨 인코더 준비
    le = LabelEncoder()

    # 타겟 타입이 범주형이거나 오브젝이면 라벨 인코딩
    if trdf['tp'].dtype == 'object' or pd.api.types.is_categorical_dtype(trdf['tp']):
        trdf['tp'] = le.fit_transform(trdf['tp'])

    # 점수용 데이터 제작
    y = trdf['tp']
    X = trdf.drop(columns=['tp'], errors='ignore')
    
    # 8:2로 나누기
    X80, X20, y80, y20 = train_test_split(X, y, test_size=0.2, random_state=8989, stratify=y)

    
    # 중요: 학습 데이터와 테스트 데이터의 컬럼을 동일하게 만들기 위해
    # 두 데이터를 합쳤다가 인코딩 후 다시 분리합니다.
    combined_df = pd.concat([X_train, X_test], keys=['train', 'test'])
    combined_df_encoded = pd.get_dummies(combined_df, columns=['color'], drop_first=True)
    
    # 다시 학습용과 테스트용으로 분리
    X_train_final = combined_df_encoded.loc['train']
    X_test_final = combined_df_encoded.loc['test']
    
    # 타겟 변수(y) 정의
    y_train_final = trdf['tp']
    
    print("학습 및 테스트 데이터 전처리 완료.")
    
    # 기본 모델 설정 및 학습
    if model is None:
        model = XGBClassifier(random_state=8989, use_label_encoder=False)
    model.fit(X80, y80)

    print("점수용 모델 학습 완료.")
    
    # 예측 및 확률 계산
    pred = model.predict(X20)
    pred_proba = model.predict_proba(X20)

    # 평가지표 계산
    num_classes = y.nunique()
    avg_method = 'binary' if num_classes == 2 else 'macro'
    
    current_scores = {
        'Accuracy': round(accuracy_score(y20, pred)),
        'Precision': round(precision_score(y20, pred, average=avg_method, zero_division=0)),
        'Recall': round(recall_score(y20, pred, average=avg_method, zero_division=0)),
        'F1_Score': round(f1_score(y20, pred, average=avg_method, zero_division=0))
    }

    # 점수 출력
    print(f"MODEL : {model.__class__.__name__}")
    for key, val in current_scores.items():
         print(f"{key:<10} : {val:.5f}")
    print("-" * 30)

    # 모델 재학습 우려로 인해 모델 초기화
    model_f = model
    
    model_f.fit(X_train_final, y_train_final)

    print("테스트 모델 학습 완료")
    
    # 학습된 모델로 테스트 데이터의 결과를 예측합니다.
    predictions_numeric = model_f.predict(X_test_final)
    
    # 예측된 숫자(0, 1, 2)를 다시 원래의 문자열('Ghoul', 'Goblin', 'Ghost')로 변환합니다.
    predictions = le.inverse_transform(predictions_numeric)
    
    print("테스트 데이터 예측 완료.")
    
    
    # ==============================================================================
    # 5. 제출 파일 생성
    # ==============================================================================
    # 캐글 제출 양식에 맞춰 DataFrame을 생성합니다.
    submission_df = pd.DataFrame({
        "id": test_ids,
        "type": predictions
    })
    
    # CSV 파일로 저장합니다. index=False는 불필요한 인덱스 컬럼이 저장되지 않도록 합니다.
    submission_df.to_csv(f"submission{name}.csv", index=False)
    
    print(f"\n✅ submission{name}.csv 파일 생성이 완료되었습니다!")
    print("제출 파일 미리보기:")
    print(submission_df.head())

SyntaxError: non-default argument follows default argument (2188701028.py, line 5)

# 이상치 처리

In [12]:
df = pd.read_csv('../data/train.csv')

numeric_cols = df.select_dtypes(include='number').columns
dft = df.copy()

for col in numeric_cols:

    Q1 = dft[col].quantile(0.25)
    Q3 = dft[col].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    dft = dft[(dft[col] >= lower_bound) & (dft[col] <= upper_bound)]

print("Original data shape:", df.shape)
print("Data shape after removing outliers:", dft.shape)

Original data shape: (371, 7)
Data shape after removing outliers: (366, 7)
